# zib gereed maken voo rmapping

* download van zibs
* inlezen en layout consistent maken met XtEHR
* opslaan voor mapping - zie mapping Notebook

## imports

In [1]:
import pandas as pd
import os

## toon de zibs in folder (hier: zib2020)

In [4]:
base = 'zibs2020'
folder = os.path.join(base, 'downloads')
files = os.listdir(folder)
files

['Nl.zorg.Alert-v4.1(2020EN).xlsx',
 'Nl.zorg.AllergyIntolerance-v3.3(2020EN).xlsx',
 'Nl.zorg.HealthcareProvider-v3.4(2020EN).xlsx',
 'Nl.zorg.HealthProfessional-v3.5(2020EN).xlsx',
 'Nl.zorg.MedicalDevice-v3.3.1(2020EN).xlsx',
 'Nl.zorg.NursingIntervention-v3.2(2020EN).xlsx',
 'Nl.zorg.Patient-v3.2(2020EN).xlsx',
 'Nl.zorg.Problem-v4.4(2020EN).xlsx',
 'Nl.zorg.Procedure-v5.2(2020EN).xlsx']

## kies een zib

In [5]:
file = 'Nl.zorg.NursingIntervention-v3.2(2020EN).xlsx'
print(file)
url = os.path.join(folder, file)
zibdf = pd.read_excel(url, sheet_name='Data', header=1)
zibdf

Nl.zorg.NursingIntervention-v3.2(2020EN).xlsx


C:\Users\edelman_nict1\AppData\Roaming\Python\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Concept,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Alias,Type,Card.,Stereotype,Id,Definition,DefinitionCode,Reference,Constraints
0,NaN,NursingIntervention,NaN,NaN,NaN,NaN,NaN,NL: VerpleegkundigeInterventie,NaN,NaN,rootconcept,NL-CM:14.2.1,Root concept of the NursingIntervention inform...,SNOMED CT: 9632001 Nursing procedure,NaN,NaN
1,NaN,NaN,Intervention,NaN,NaN,NaN,NaN,NL: Interventie,CD,1,data,NL-CM:14.2.2,A nursing intervention is a treatment carried ...,NaN,InterventionNICCodelist,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,InterventionSnomedCodelist,NaN
3,NaN,NaN,Indication::Problem,NaN,NaN,NaN,NaN,NL: Indicatie::Probleem,NaN,0..1,"data,reference",NL-CM:14.2.6,The nursing-related problem (indication) provi...,NaN,This is a reference to the rootconcept of info...,NaN
4,NaN,NaN,TreatmentObjective,NaN,NaN,NaN,NaN,NL: Behandeldoel,NaN,0..1,"data,reference",NL-CM:14.2.14,The description of the treatment goal that the...,NaN,This is a reference to the rootconcept of info...,NaN
5,NaN,NaN,Interval,NaN,NaN,NaN,NaN,NL: Interval,PQ,0..1,data,NL-CM:14.2.3,Interval indicates the time between planned pr...,NaN,NaN,NaN
6,NaN,NaN,Frequency,NaN,NaN,NaN,NaN,NL: Frequentie,PQ,0..1,data,NL-CM:14.2.4,The frequency describes how often and in which...,NaN,NaN,NaN
7,NaN,NaN,ProcedureStartDateTime,NaN,NaN,NaN,NaN,NL: ActieStartDatumTijd,TS,0..1,data,NL-CM:14.2.11,The start date (and if possible start time) of...,NaN,NaN,NaN
8,NaN,NaN,ProcedureEndDateTime,NaN,NaN,NaN,NaN,NL: ActieEindDatumTijd,TS,0..1,data,NL-CM:14.2.12,The end date (and if possible end time) of the...,NaN,NaN,NaN
9,NaN,NaN,MedicalDevice,NaN,NaN,NaN,NaN,NL: MedischHulpmiddel,NaN,0..*,"data,reference",NL-CM:14.2.13,Description of the materials used for the nurs...,NaN,This is a reference to the rootconcept of info...,NaN


## opschonen zib

* lowercase kolomnamen
* paar labels laten vervallen
* name en path kolom toevoegen
* name vullen (naam attribuut)
* path vullen (als in logical model, dus pad.pad2.pad3.naam etc.)
* kolommen opschonen tot benodigde

In [6]:
zibdf.columns = [col.lower() for col in zibdf.columns]
zibdf = zibdf.drop(labels=['unnamed: 0', 'reference'], axis=1).dropna(how='all')
zibdf.insert(0, 'name', None)
zibdf.insert(0, 'path', None)
zibdf.fillna('')
concepts = []
for i, row in zibdf.iterrows():
    # Loop thru columns 2 tot 8 (Concept and 'Unnamed' ones)
    for j in range(2, 6):
        # print(j, row.iloc[j])
        if not pd.isna(row.iloc[j]):
            # Remove last element if next column is empty
            if len(concepts) > j -2:
                concepts.pop()
            try:
                concepts[j - 2] = row.iloc[j]
            except IndexError:
                concepts.append(row.iloc[j])
    # print(i, concepts)
    zibdf.at[i, 'path'] = '.'.join(concepts)
    zibdf.at[i, 'name'] = concepts[-1]
zibdf = zibdf[['name', 'path', 'alias', 'type', 'card.', 'stereotype', 'id', 'definition', 'definitioncode']].fillna('')
zibdf

,name,path,alias,type,card.,stereotype,id,definition,definitioncode
0,NursingIntervention,NursingIntervention,NL: VerpleegkundigeInterventie,,,rootconcept,NL-CM:14.2.1,Root concept of the NursingIntervention inform...,SNOMED CT: 9632001 Nursing procedure
1,Intervention,NursingIntervention.Intervention,NL: Interventie,CD,1,data,NL-CM:14.2.2,A nursing intervention is a treatment carried ...,
3,Indication::Problem,NursingIntervention.Indication::Problem,NL: Indicatie::Probleem,,0..1,"data,reference",NL-CM:14.2.6,The nursing-related problem (indication) provi...,
4,TreatmentObjective,NursingIntervention.TreatmentObjective,NL: Behandeldoel,,0..1,"data,reference",NL-CM:14.2.14,The description of the treatment goal that the...,
5,Interval,NursingIntervention.Interval,NL: Interval,PQ,0..1,data,NL-CM:14.2.3,Interval indicates the time between planned pr...,
6,Frequency,NursingIntervention.Frequency,NL: Frequentie,PQ,0..1,data,NL-CM:14.2.4,The frequency describes how often and in which...,
7,ProcedureStartDateTime,NursingIntervention.ProcedureStartDateTime,NL: ActieStartDatumTijd,TS,0..1,data,NL-CM:14.2.11,The start date (and if possible start time) of...,
8,ProcedureEndDateTime,NursingIntervention.ProcedureEndDateTime,NL: ActieEindDatumTijd,TS,0..1,data,NL-CM:14.2.12,The end date (and if possible end time) of the...,
9,MedicalDevice,NursingIntervention.MedicalDevice,NL: MedischHulpmiddel,,0..*,"data,reference",NL-CM:14.2.13,Description of the materials used for the nurs...,
10,Instruction,NursingIntervention.Instruction,NL: Instructie,ST,0..1,data,NL-CM:14.2.18,Instructions for performing the nursing action...,


## Excel opslaan

In [7]:
url = os.path.join(base, 'mappable', file)
zibdf.to_excel(url, index=False)